In [7]:

# aws s3 ls s3://prod-marketplace-etl-raw-data-transformed/2019-12-03/product-support-queries.json --human-readable


# import boto3
# bucketname='prod-marketplace-etl-raw-data-transformed'
# key='2019-12-03/product-support-queries.json'
# s3_resource=boto3.client('s3').get_object()
# print(s3_resource.Bucket(bucketname).Object(key).content_length)
# #.Bucket(bucketname).Object(keyname).content_length

import boto3
import json
import os

from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def s3_exists(bucketname,key):
    s3=boto3.client('s3')
    """return the key's size if it exist, else None"""
    response = s3.list_objects_v2(
        Bucket=bucketname,
        Prefix=key
    )
    for obj in response.get('Contents', []):
        if obj['Key'] == key:
            return True
    return False


def show(data,method):
    if method=="print":
        print(data)
    else:
        f_name=method.split(":")[0]
        mode=method.split(":")[1]
        with open(f_name,mode) as f:
            f.write(data+"\n")


    

s3_RES=boto3.resource('s3')
s3_CLIENT=boto3.client('s3')


def download_s3_content_string(bucketname,key):
    response=s3_CLIENT.get_object(Bucket=bucketname, Key=key) 
    content=response["Body"].read().decode()
    return content
        
def flatten(d, parent_key='', sep='.',listaction=None):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep,listaction=listaction).items())
        # EACH LIST WILL HAVE ITS OWN FLATTENING LOGIC 
        elif isinstance(v,list):
            if listaction is not None:
                items.extend(listaction(list_val=v, parent=new_key, sep=sep).items())
            else:
                print("Listaction is None, but there is a list")
                print(v)
        
        else:
            items.append((new_key, v))
    return dict(items)

    




# TEST S3 FILES EXISTANCE


In [2]:
def get_s3_files(file_name,start_date,end_date, bucketname,print_sample=False ):
    output_file_name=file_name.split(".")[0]+".md"    
    if os.path.exists(output_file_name) and os.path.isfile(output_file_name):
        os.remove(output_file_name)
    for single_date in daterange(start_date, end_date):
        d=single_date.strftime("%Y-%m-%d")
        key='{}/{}'.format(d,file_name)
        if s3_exists(bucketname,key):
            s3_obj=s3_RES.Object(bucketname,key)  
            output="{}:{}".format(output_file_name,"a") 
            show("# {}".format(key),output)
            show("\n+ {}: {} kb: s3://{}/{}".format(d,s3_obj.content_length,s3_obj.bucket_name,s3_obj.key),output)
            res=s3_CLIENT.get_object(Bucket=bucketname, Key=key) 
            vis_json="~~~\n{}\n~~~".format(res["Body"].read().decode())
            if print_sample:
                show(vis_json,output)
        else: 
            output="{}:{}".format(output_file_name,"a")
            show("# {}".format(key),output)
        


# GET SUMMARY AND HEAD OF FILES


In [9]:
# MAIN AREA
env='prod'
bucketname='{}-marketplace-etl-raw-data-to-process'.format(env)
#bucketname='{}-marketplace-etl-raw-data-transformed'.format(env)
start_date = date(2019, 12, 2)
end_date = date(2019, 12, 3)
#r=['adwords_ads_campaign_performance', 'adwords_all_campaign_performance', 'adwords_campaigns', 'ga_pages', 'landing_page_stats', 'mpac_addon_convs', 'mpac_landing_page_addon_stats', 'mpac_landing_page_convs', 'mpac_page_addon_stats', 'mpac_tab_convs', 'page_stats' ]

#file_name='adwords_all_campaign_performance'
sources=['jira-cssales']


for dt in daterange(start_date,end_date):
    for source in sources:
        j_string=download_s3_content_string(bucketname,f"{dt}/{source}.json")
        with open(f"data/{dt}_{source}.json",'w') as f:
            f.write(j_string)
            

# FIND OUT S3 FILE CONTENT SPECIFICS

## SEARCH FROM DOWNLOADED FILE

In [144]:
import json 


from datetime import timedelta, date

def downloaded_transform_to_json_objects(string_input):
    return json.loads(string_input)

env='prod'
start_date=date(2019,12,15)
end_date=date(2019,12,21)
file_name='licenses.json'
bucketname=f'{env}-marketplace-etl-raw-data-to-process'
dict_of_json_down={}
dict_of_string_down={}
for single_date in daterange(start_date, end_date):
    ky_name=f'{single_date}/{file_name}'
    print(ky_name)
    tmp=download_s3_content_string(bucketname=bucketname,key=f'{single_date}/{file_name}')
    dict_of_json_down[ky_name.replace("/","_")]=downloaded_transform_to_json_objects(tmp)
    dict_of_string_down[ky_name.replace("/","_")]=tmp



2019-12-15/licenses.json
2019-12-16/licenses.json
2019-12-17/licenses.json
2019-12-18/licenses.json
2019-12-19/licenses.json
2019-12-20/licenses.json


### Explore the structure of the downloaded file

The downloaded file is likely gonna be a list of objects

In [142]:
from pprint import pprint
for k,v in dict_of_json_down.items():
    print(k)
    first_level_keys={}
    for json_obj in v: 
        f_j=flatten(json_obj)
        for k in f_j.keys():
            first_level_keys[k]=""
    pprint(first_level_keys)
        

2019-12-15_licenses-attributions.json
{'addonKey': '',
 'addonLicenseId': '',
 'addonName': '',
 'attribution.campaignContent': '',
 'attribution.campaignMedium': '',
 'attribution.campaignName': '',
 'attribution.campaignSource': '',
 'attribution.channel': '',
 'contactDetails.billingContact.address1': '',
 'contactDetails.billingContact.address2': '',
 'contactDetails.billingContact.city': '',
 'contactDetails.billingContact.email': '',
 'contactDetails.billingContact.name': '',
 'contactDetails.billingContact.phone': '',
 'contactDetails.billingContact.postcode': '',
 'contactDetails.billingContact.state': '',
 'contactDetails.company': '',
 'contactDetails.country': '',
 'contactDetails.region': '',
 'contactDetails.technicalContact.address1': '',
 'contactDetails.technicalContact.address2': '',
 'contactDetails.technicalContact.city': '',
 'contactDetails.technicalContact.email': '',
 'contactDetails.technicalContact.name': '',
 'contactDetails.technicalContact.phone': '',
 'cont

### SEARCH FOR THE SPECIFIED STRING 

In [143]:
import json
# JSONPATH TUTORIAL THIS IS BASED ON: https://www.journaldev.com/33265/python-jsonpath-examples
from jsonpath_ng import jsonpath, parse
jsonpath_expression_str='$.attribution'
jsonpath_expression=parse(jsonpath_expression_str)
matches={}
for downloaded_date,json_arr in dict_of_json_down.items():
    results=[]
    for json_obj in json_arr:
        match=jsonpath_expression.find(json_obj)
        if len(match)!=0:
            results.append(match)
    matches[downloaded_date]=results 
    
for k,v in matches.items():
    print(f'{k} has {len(v)} attribution fields')

2019-12-15_licenses-attributions.json has 15422 attribution fields
2019-12-16_licenses-attributions.json has 15395 attribution fields
2019-12-17_licenses-attributions.json has 0 attribution fields
2019-12-18_licenses-attributions.json has 0 attribution fields
2019-12-19_licenses-attributions.json has 0 attribution fields
2019-12-20_licenses-attributions.json has 0 attribution fields


In [104]:
import json
for k,v in dict_of_string_down.items():
    with open(f"data/{k}","w") as f:
        f.writelines(v)


## SEARCH FROM TRANSFORMED FILE


In [100]:
import json 
def transform_to_json_objects(string_input):
    j_res=string_input.split('\n')
    return list(map(lambda x: json.loads(x),j_res))
      
from datetime import timedelta, date
env='prod'
start_date=date(2019,11,20)
end_date=date(2019,12,21)
file_name='licenses-attributions.json'
bucketname=f'{env}-marketplace-etl-raw-data-transformed'
dict_of_json={}
dict_of_string={}

for single_date in daterange(start_date, end_date):
    ky_name=f'{single_date}/{file_name}'
    print(ky_name)
    tmp=download_s3_content_string(bucketname=bucketname,key=f'{single_date}/{file_name}')
    dict_of_json[ky_name.replace("/","_")]=transform_to_json_objects(tmp)
    dict_of_string[ky_name.replace("/","_")]=tmp

2019-11-20/licenses-attributions.json
2019-11-21/licenses-attributions.json
2019-11-22/licenses-attributions.json
2019-11-23/licenses-attributions.json
2019-11-24/licenses-attributions.json
2019-11-25/licenses-attributions.json
2019-11-26/licenses-attributions.json
2019-11-27/licenses-attributions.json
2019-11-28/licenses-attributions.json
2019-11-29/licenses-attributions.json
2019-11-30/licenses-attributions.json
2019-12-01/licenses-attributions.json
2019-12-02/licenses-attributions.json
2019-12-03/licenses-attributions.json
2019-12-04/licenses-attributions.json
2019-12-05/licenses-attributions.json
2019-12-06/licenses-attributions.json
2019-12-07/licenses-attributions.json
2019-12-08/licenses-attributions.json
2019-12-09/licenses-attributions.json
2019-12-10/licenses-attributions.json
2019-12-11/licenses-attributions.json
2019-12-12/licenses-attributions.json
2019-12-13/licenses-attributions.json
2019-12-14/licenses-attributions.json
2019-12-15/licenses-attributions.json
2019-12-16/l

In [97]:
import json
for k,v in dict_of_string.items():
    with open(f"data/{k}","w") as f:
        f.writelines(v)

    

In [101]:
import json
# JSONPATH TUTORIAL THIS IS BASED ON: https://www.journaldev.com/33265/python-jsonpath-examples
from jsonpath_ng import jsonpath, parse
jsonpath_expression_str='$.attribution'
jsonpath_expression=parse(jsonpath_expression_str)
matches={}
for downloaded_date,json_arr in dict_of_json.items():
    results=[]
    for json_obj in json_arr:
        match=jsonpath_expression.find(json_obj)
        if len(match)!=0:
            results.append(match)
    matches[downloaded_date]=results 
    

for k,v in matches.items():
    print(f'{k} has {len(v)} attribution fields')
    

2019-11-20_licenses-attributions.json has 10995 attribution fields
2019-11-21_licenses-attributions.json has 10824 attribution fields
2019-11-22_licenses-attributions.json has 10827 attribution fields
2019-11-23_licenses-attributions.json has 10986 attribution fields
2019-11-24_licenses-attributions.json has 11117 attribution fields
2019-11-25_licenses-attributions.json has 11079 attribution fields
2019-11-26_licenses-attributions.json has 11002 attribution fields
2019-11-27_licenses-attributions.json has 10804 attribution fields
2019-11-28_licenses-attributions.json has 10650 attribution fields
2019-11-29_licenses-attributions.json has 10693 attribution fields
2019-11-30_licenses-attributions.json has 10857 attribution fields
2019-12-01_licenses-attributions.json has 10988 attribution fields
2019-12-02_licenses-attributions.json has 10960 attribution fields
2019-12-03_licenses-attributions.json has 10841 attribution fields
2019-12-04_licenses-attributions.json has 10659 attribution fi

In [96]:
for rs in results[:100]:
    print(rs[0].value)

{'channel': 'Atlassian'}
{'channel': 'Direct'}
{'channel': 'Atlassian'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Atlassian'}
{'channel': 'Paid Search', 'campaignName': 'P:jira-software|O:ppm|V:google|G:seas|L:en|F:consider|S:brand|M:bmm|A:text|D:desktop', 'campaignSource': 'google', 'campaignMedium': 'cpc'}
{'channel': 'Paid Search', 'campaignName': 'P:jira-software|O:ppm|V:google|G:seas|L:en|F:consider|S:brand|M:bmm|A:text|D:desktop', 'campaignSource': 'google', 'campaignMedium': 'cpc'}
{'channel': 'Direct'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Atlassian'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channel': 'Organic search'}
{'channe